In [14]:
import numpy as np
import sympy as sp
from scipy.sparse import coo_matrix, csr_matrix   #for sparse matrix
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from scipy.sparse.linalg import spsolve
np.set_printoptions(
    precision=4,
    formatter={'float': '{:.4e}'.format}
)


Create routines for 4 node element, large deformations

In [ ]:
#define isoparametric coordinates and shape functions:
xi = xi1,xi2 = sp.symbols('xi1 xi2',real=True)
N1 = 0.25*(xi[0] - 1.)*(xi[1] - 1.); N2 = -0.25*(xi[0] + 1.)*(xi[1] - 1.);
N3 = 0.25*(xi[0]  +1.)*(xi[1] + 1.); N4 = -0.25*(xi[0] - 1.)*(xi[1] + 1.);
Ne=sp.Matrix([[N1,0,N2,0,N3,0,N4,0],[0,N1,0,N2,0,N3,0,N4]])

#generate function, and save for later use
Ne_quad_largedef_func=sp.lambdify([xi],Ne )
#import dill
#dill.settings['recurse'] = True
#dill.dump(Ne_quad_largedef_func, open("Ne_quad_largedef_func_dill", "wb"))
#To load this file later use: Ne_quad_func=dill.load(open("Ne_quad_func_dill", "rb"))  

#differentiate shape functions wrt isoparam. coordinates 
dN1_dxi=sp.simplify(sp.Matrix( [sp.diff(N1,xi[0]),sp.diff(N1,xi[1])] ));
dN2_dxi=sp.simplify(sp.Matrix( [sp.diff(N2,xi[0]),sp.diff(N2,xi[1])] ));
dN3_dxi=sp.simplify(sp.Matrix( [sp.diff(N3,xi[0]),sp.diff(N3,xi[1])] ));
dN4_dxi=sp.simplify(sp.Matrix( [sp.diff(N4,xi[0]),sp.diff(N4,xi[1])] ));

#introduce initial node positions via Ex, Ey
Xe1 = sp.Matrix(sp.symbols('Xe1_1:3', real=True))
Xe2 = sp.Matrix(sp.symbols('Xe2_1:3', real=True))
Xe3 = sp.Matrix(sp.symbols('Xe3_1:3', real=True))
Xe4 = sp.Matrix(sp.symbols('Xe4_1:3', real=True))

#introduce initial coordinate as fcn of isoparam. coord.
X=N1*sp.Matrix([Xe1[0],Xe1[1]])+N2*sp.Matrix([Xe2[0],Xe2[1]])+N3*sp.Matrix([Xe3[0],Xe3[1]])+N4*sp.Matrix([Xe4[0],Xe4[1]])

#compute Jacobian and its determinant, transpose and inverse of transpose
Fisop=X.jacobian(xi)
detFisop=Fisop.det()
FisopT=sp.Transpose(Fisop)
invFisopT=sp.simplify(sp.Inverse(FisopT))

#generate function and save it for later use
detFisop_quad_largedef_func=sp.lambdify([xi,Xe1,Xe2,Xe3,Xe4],detFisop)
#dill.dump(detFisop_quad_largedef_func, open("detFisop_quad_largedef_func_dill", "wb"))

#use chain rule to compute spatial derivatives
dN1_dX=invFisopT*dN1_dxi 
dN2_dX=invFisopT*dN2_dxi 
dN3_dX=invFisopT*dN3_dxi 
dN4_dX=invFisopT*dN4_dxi

#define element B0-matrix
Be0=sp.Matrix([[dN1_dX[0], 0, dN2_dX[0], 0, dN3_dX[0], 0, dN4_dX[0], 0],
           [0, dN1_dX[1], 0, dN2_dX[1], 0, dN3_dX[1], 0, dN4_dX[1]],
           [dN1_dX[1], 0, dN2_dX[1], 0, dN3_dX[1], 0, dN4_dX[1],0], 
           [0,dN1_dX[0], 0, dN2_dX[0], 0, dN3_dX[0], 0, dN4_dX[0]]])

Be0_quad_largedef_func=sp.lambdify([xi,Xe1,Xe2,Xe3,Xe4],Be0 )
#dill.dump(Be0_quad_largedef_func, open("Be0_quad_largedef_func_dill", "wb"))

Test routines

In [ ]:
#define node positions via Ex and Ey
Ex=np.array([0.,3.,4.,1.])
Ey=np.array([0.,0.,5.,3.])
Xe1=np.array([Ex[0],Ey[0]])
Xe2=np.array([Ex[1],Ey[1]])
Xe3=np.array([Ex[2],Ey[2]])
Xe4=np.array([Ex[3],Ey[3]])
#integration point location 
xi=np.array([0.1,0.1])

Be0=Be0_quad_largedef_func(xi,Xe1,Xe2,Xe3,Xe4)
print(Be0)
detFisop=detFisop_quad_largedef_func(xi,Xe1,Xe2,Xe3,Xe4)
print(detFisop)
Ne=Ne_quad_largedef_func(xi)
print(Ne)

In [24]:
#another test
#define node positions via Ex and Ey
Ex=np.array([-1.,1.,1.,-1.])
Ey=np.array([-1.,-1.,1.,1.])
Xe1=np.array([Ex[0],Ey[0]])
Xe2=np.array([Ex[1],Ey[1]])
Xe3=np.array([Ex[2],Ey[2]])
Xe4=np.array([Ex[3],Ey[3]])
#evaluation point location 
xi=np.array([0.,0.])

Be0=Be0_quad_largedef_func(xi,Xe1,Xe2,Xe3,Xe4)
#print(Be0)
#assume element displacements
ae=np.array([0.01,0.02,-0.03,0.02, -0.04, 0.06, 0.01, 0.00])
F2d=np.array([1.,1.,0.,0.])+Be0@ae
print(F2d)

[9.7750e-01 1.0050e+00 -2.5000e-03 1.5000e-02]


Create material model routine, Neo-Hooke

In [23]:
#define 2d deformation gradient in Voigt format:
Fv = sp.symbols('Fv1:5',real=True)
#transform to 3x3 format
F=sp.Matrix([[Fv[0],Fv[2],0],[Fv[3],Fv[1],0],[0,0,1]])
C=F.T * F
invC=C.inv()
J=F.det()
Gmod,lambdamod=sp.symbols('Gmod,lambdamod')
S=Gmod*(sp.eye(3)-invC)+lambdamod*sp.log(J)*invC
P=F * S
#transform to 2d Voigt format
Pv=sp.Matrix([P[0,0],P[1,1],P[0,1],P[1,0]])
P_NH_func= sp.lambdify([Fv, Gmod, lambdamod], Pv, modules='numpy')
#compute stiffness
dPvdFv=Pv.jacobian(Fv)
#generate function and save for later use
dPdF_NH_func=sp.lambdify([Fv,Gmod,lambdamod],dPvdFv,modules='numpy')

Routine for precomputing matrices (to gain efficiency)

In [ ]:
def create_Be0_detFiso_matrix(nel, Ex, Ey):
    
    ngp=4
    Be0_matrix = np.zeros((nel,ngp, ngp, 8))
    detFisop_matrix = np.zeros((nel, ngp))
    nn=1./np.sqrt(3.)
    xi_v=np.array([[-nn,-nn,nn,nn],[-nn,nn,-nn,nn]])
    H_v=np.array([1.,1.,1.,1.])

    for el in range(nel):
        Xe1 = np.array([Ex[el,0], Ey[el,0]])
        Xe2 = np.array([Ex[el,1], Ey[el,1]])
        Xe3 = np.array([Ex[el,2], Ey[el,2]])
        Xe4 = np.array([Ex[el,3], Ey[el,3]])

        for gp in range(ngp):
            xi = np.array([xi_v[0,gp], xi_v[1,gp]])
            Be0 = Be0_quad_largedef_func(xi, Xe1, Xe2, Xe3, Xe4)
            Be0_matrix[el, gp, :, :] = Be0
            detFisotr=detFisop_quad_largedef_func(xi,Xe1,Xe2,Xe3,Xe4)
            detFisop_matrix[el, gp] = detFisotr

    return Be0_matrix, detFisop_matrix


Define element using precomputed matrices

In [ ]:
def quad_large_def_element(el,ae,Be0_matrix,detFisop_matrix,mtrlpar,thickness): #(ae,Ex,Ey,mtrlpar,thickness):
    #Gauss-points and weights
    ngp=4
    nn=1./np.sqrt(3.)
    xi_v=np.array([[-nn,-nn,nn,nn],[-nn,nn,-nn,nn]])
    H_v=np.array([1.,1.,1.,1.])
    #initialize output
    #F2d_m=np.zeros((4,ngp))
    #Pv_m=np.zeros((4,ngp))
    #dPvdFv_m=np.zeros((4,4,ngp))
    #initialize internal element force and stiffness
    fe_int=np.zeros(8); Ke_int=np.zeros((8,8))
    for j in range(ngp):
        Be0=Be0_matrix[el, j, :, :]  #Be0_quad_largedef_func(xi,Ex,Ey)
        detFisop=detFisop_matrix[el, j] #detFisop_quad_largedef_func(xi,Ex,Ey)
        #2d deformation gradient
        F2d=np.array([1.,1.,0.,0.])+Be0@ae
        P=(P_NH_func(F2d,mtrlpar[0],mtrlpar[1])).flatten() #Here Neo Hooke
        dPdFv=dPdF_NH_func(F2d,mtrlpar[0],mtrlpar[1])    #Here Neo Hooke
        fe_int+=Be0.T@P*detFisop*thickness*H_v[j]
        Ke_int+=(Be0.T@dPdFv@Be0)*detFisop*thickness*H_v[j]
    return (fe_int,Ke_int) #,F2d_m,Pv_m,dPvdFv_m)


Test of element

In [ ]:
#define node positions via Ex and Ey
Ex=np.array([[-1.,1.,1.,-1.]])
Ey=np.array([[-1.,-1.,1.,1.]])

#assume element displacements
ae=np.array([0.1,0.2,-0.3,0.2, -0.4, 0.6, 0.1, 0.15])

#material data
Gmod = 3.
lambdamod = 2.
mtrlpar=np.array([Gmod,lambdamod])
#thickness
thickness=1.1


Be0_matrix, detFisop_matrix=create_Be0_detFiso_matrix(1, Ex, Ey)
fe_int,Ke_int=quad_large_def_element(0,ae,Be0_matrix,detFisop_matrix,mtrlpar,thickness)
print(fe_int)
print(Ke_int[:,:])
